# 5장 YOLO v11 Training

In [1]:
!pip install -qU ultralytics roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 102.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3

In [2]:
# PyYAML roboflow

import os
import requests
import numpy as np
import ultralytics
import yaml
import zipfile

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## 1. License Plate Recognition Computer Vision Project: 단일객체 검출

### License Plate dataset download from roboflow
- https://universe.roboflow.com

In [9]:
## 1. Using roboflow api
from roboflow import Roboflow
rf = Roboflow(api_key="HHKXlDyiIfBO33SpID7d")
project = rf.workspace("roboflow-universe-projects").project("license-plate-recognition-rxg4e")
version = project.version(11)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to License-Plate-Recognition-11 in yolov11:: 100%|██████████| 20262/20262 [00:02<00:00, 7727.92it/s]


### YOLO 객체 검출 model import

In [20]:
model = ultralytics.YOLO('yolo11n.pt')

print(model.names) # pretrained model trained using coco dataset

{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

### YOLO 객체 검출 yaml 파일 만들기

In [28]:
data = {
    'train' :'/content/License_plate/train/images',
    'val' : '/content/License_plate/valid/images',
    'test' :'/content/License_plate/test/images',
    'names' : ["License_plate"],
    'nc' : 1
}

with open('/content/License_plate/data.yaml', 'w') as f:
    yaml.dump(data, f)

### Yolo v11 License plate 객체검출 학습

In [29]:
model.train(data = '/content/License_plate/data.yaml', epochs = 5, patience = 5, imgsz = 256)

# 훈련 지표 (Training Metrics)
# GPU_mem (0.516G): 현재 모델이 사용하고 있는 GPU(그래픽 처리 장치) 메모리 양.
# box_loss (1.365): 바운딩 박스 회귀와 관련된 손실 값. 예측된 바운딩 박스가 실제 바운딩 박스와 위치, 너비, 높이 면에서 얼마나 잘 일치하는지를 정량화
# cls_loss (1.328): 분류 손실. 모델이 탐지된 각 객체에 대해 올바른 클래스를 얼마나 잘 예측하는지를 측정
# dfl_loss (1.056): DFL(Distribution Focal Loss)의 약자입니다. 이 손실은 현대 객체 탐지 모델(일부 YOLO 버전과 같은)에서 모델이 분류하기 어려운 예시에 집중
# Instances (1): 현재 반복(iteration)에서 처리된 인스턴스 또는 배치(batch)의 수
# Size (256): 모델이 처리하는 입력 이미지의 크기(예: 256x256 픽셀)입니다.

# 평가 지표 (Evaluation Metrics)
# Images (2047): 검증/평가 데이터셋에 있는 총 이미지 수
# Instances (2194): 검증/평가 데이터셋의 모든 이미지에 걸쳐 있는 실제 객체 인스턴스(바운딩 박스)의 총 수
# Box (P: Precision, R: Recall, mAP50, mAP50-95):
# P (Precision, 정밀도): 모델이 '있다'고 예측한 것 중 실제로 맞는 것의 비율
# R (Recall, 재현율): 실제로 있어야 할 객체 중 모델이 얼마나 많이 찾아냈는지를 나타냄
# mAP50 (평균 정밀도, IoU 0.5): IoU(Intersection over Union, 겹침 영역 비율) 임계값을 0.5로 설정했을 때의 평균 정밀도
# mAP50-95 (평균 정밀도, IoU 0.5~0.95): IoU 임계값을 0.5부터 0.95까지 0.05 단위로 증가시키면서 계산한 평균 정밀도들의 평균

Ultralytics 8.3.172 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/License_plate/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train6, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=5, perspective=0.0, plots=True, pose=12.0, pretrai

train: Scanning /content/License_plate/train/labels... 7057 images, 5 backgrounds, 0 corrupt: 100%|██████████| 7057/7057 [00:04<00:00, 1547.34it/s]


train: New cache created: /content/License_plate/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 294.1±94.9 MB/s, size: 20.8 KB)


val: Scanning /content/License_plate/valid/labels... 2048 images, 3 backgrounds, 0 corrupt: 100%|██████████| 2048/2048 [00:01<00:00, 1415.79it/s]

val: New cache created: /content/License_plate/valid/labels.cache


...

                   all       2048       2195      0.968      0.894      0.932      0.633



5 epochs completed in 0.069 hours.
Optimizer stripped from runs/detect/train6/weights/last.pt, 5.4MB
Optimizer stripped from runs/detect/train6/weights/best.pt, 5.4MB

Validating runs/detect/train6/weights/best.pt...
Ultralytics 8.3.172 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 64/64 [00:07<00:00,  8.09it/s]


                   all       2048       2195      0.968      0.894      0.932      0.634
Speed: 0.0ms preprocess, 0.4ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train6


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ddec6632690>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### License_plate 학습 모델 불러오기

In [33]:
model_path ='/content/runs/detect/train6/weights/best.pt'
model = ultralytics.YOLO(model_path)

In [35]:
result = model(source = '/content/License_plate/test/images', save = True)


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1020 /content/License_plate/test/images/0002a5b67e5f0909_jpg.rf.c8f81ef986e3e99af6f349c200080453.jpg: 192x256 1 License_plate, 69.1ms
image 2/1020 /content/License_plate/test/images/000812dcf304a8e7_jpg.rf.ba32e6c184b3d974abcced6f7c29af6d.jpg: 224x256 2 License_plates, 71.0ms
image 3/1020 /content/License_plate/test/images/0010f4c10f7ab07e_jpg.rf.1844f6dde3b97ed1c762db933bbacaf3.jpg: 192x256 1 License_plate, 10.7ms
image 4/1020 /content/License

In [37]:
for i, r in enumerate(result):
    boxes = r.boxes  # Boxes object for bbox outputs
    img_id = r.path

    print("boxes = \n", boxes)
    print("img_id = \n", img_id)
    # print("probs = \n", probs)
    if i >= 2:
        break

# cls: 탐지된 객체의 예측된 클래스 ID
# conf: 탐지된 바운딩 박스가 실제로 객체를 포함하고 있다는 확신 점수(confidence score) 또는 확률
# data: (xmin, ymin, xmax, ymax, conf, cls)
# id, is_track: 객체 추적(tracking)에 사용되는 속성
# orig_shape: 입력 이미지의 원래 크기(높이, 너비)
# shape: data 텐서의 크기
# xywh: 바운딩 박스의 좌표를 (중심 x좌표, 중심 y좌표, 너비, 높이) 형식으로 픽셀 단위

boxes = 
 ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.], device='cuda:0')
conf: tensor([0.6321], device='cuda:0')
data: tensor([[313.8524, 500.9481, 422.4263, 532.6832,   0.6321,   0.0000]], device='cuda:0')
id: None
is_track: False
orig_shape: (768, 1024)
shape: torch.Size([1, 6])
xywh: tensor([[368.1394, 516.8156, 108.5739,  31.7351]], device='cuda:0')
xywhn: tensor([[0.3595, 0.6729, 0.1060, 0.0413]], device='cuda:0')
xyxy: tensor([[313.8524, 500.9481, 422.4263, 532.6832]], device='cuda:0')
xyxyn: tensor([[0.3065, 0.6523, 0.4125, 0.6936]], device='cuda:0')
img_id = 
 /content/License_plate/test/images/0002a5b67e5f0909_jpg.rf.c8f81ef986e3e99af6f349c200080453.jpg
boxes = 
 ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0., 0.], device='cuda:0')
conf: tensor([0.6119, 0.2969], device='cuda:0')
data: tensor([[3.9885e+02, 7.0816e+02, 5.0407e+02, 7.5183e+02, 6.1189e-01, 0.0000e+00],
        [3.9852e+02, 7.1122e+02, 5.0406e+02, 7.6642e+02,

## 2.Rock-paper-scissors Computer Vision Project: 다중 객체

In [40]:
import os
import requests
import numpy as np
import ultralytics
import yaml
import zipfile

### rock-paper-scissors dataset download from roboflow
- https://universe.roboflow.com/

In [42]:
## Using roboflow api
from roboflow import Roboflow
rf = Roboflow(api_key="HHKXlDyiIfBO33SpID7d")
project = rf.workspace("roboflow-58fyf").project("rock-paper-scissors-sxsw")
version = project.version(14)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to rock-paper-scissors-14 in yolov11:: 100%|██████████| 14682/14682 [00:01<00:00, 9359.11it/s] 


### YOLO 객체 검출 model import

In [43]:
model = ultralytics.YOLO('yolo11n.pt')

print(model.names) # pretrained model trained using coco dataset

{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

In [49]:
data = {
    'train' : '/content/rock-paper-scissors/train/images',
    'val' : '/content/rock-paper-scissors/valid/images',
    'test' : '/content/rock-paper-scissors/test/images',
    'names' : ['Paper', 'Rock', 'Scissors'],
    'nc' : 3
}

with open('/content/rock-paper-scissors/data.yaml', 'w') as f:
    yaml.dump(data, f)

### Yolo v11 rock-paper-scissors 객체검출 학습

In [50]:
model.train(data = '/content/rock-paper-scissors/data.yaml', epochs = 5, patience = 5, imgsz = 256)

Ultralytics 8.3.172 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/rock-paper-scissors/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=5, perspective=0.0, plots=True, pose=12.0, p

train: Scanning /content/rock-paper-scissors/train/labels... 6455 images, 2516 backgrounds, 0 corrupt: 100%|██████████| 6455/6455 [00:03<00:00, 1779.15it/s]


train: New cache created: /content/rock-paper-scissors/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 351.3±185.4 MB/s, size: 32.0 KB)


val: Scanning /content/rock-paper-scissors/valid/labels... 576 images, 238 backgrounds, 0 corrupt: 100%|██████████| 576/576 [00:00<00:00, 1378.09it/s]

val: New cache created: /content/rock-paper-scissors/valid/labels.cache


...

                   all        576        400      0.873      0.806      0.889      0.653



5 epochs completed in 0.056 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 5.4MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 5.4MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics 8.3.172 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:02<00:00,  8.32it/s]


                   all        576        400      0.876      0.808      0.889      0.654
                 Paper        132        139      0.783      0.842      0.849      0.625
                  Rock        121        141      0.887      0.834      0.908      0.649
              Scissors        116        120      0.957      0.747      0.909      0.686
Speed: 0.0ms preprocess, 0.4ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/train3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ddeadfbca90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

### rock-paper-scissors 학습 모델 불러오기

In [54]:
model_path = '/content/runs/detect/train3/weights/best.pt'
model = ultralytics.YOLO(model_path)

print(model.names)

{0: 'Paper', 1: 'Rock', 2: 'Scissors'}


In [55]:
result = model(source = '/content/rock-paper-scissors/test/images', save = True)


image 1/304 /content/rock-paper-scissors/test/images/10e0gvm_jpg.rf.3b68a834fab647f30a57fc3ea92d4cd2.jpg: 256x256 (no detections), 11.1ms
image 2/304 /content/rock-paper-scissors/test/images/15208484cellblock_jpg.rf.95cbda1e169a66105fbf2aa22959a73b.jpg: 256x256 (no detections), 9.6ms
image 3/304 /content/rock-paper-scissors/test/images/19171_298_298_1_0_jpg.rf.0024dfb25d7b5a13a78e94fca47ef004.jpg: 256x256 (no detections), 9.6ms
image 4/304 /content/rock-paper-scissors/test/images/20061004021831_jpg.rf.8667d8aa5599deb901289c024eed4313.jpg: 256x256 1 Rock, 9.6ms
image 5/304 /content/rock-paper-scissors/test/images/20220216_221550_jpg.rf.02a071a383151953fcf8671fc7fca3af.jpg: 256x256 1 Paper, 1 Scissors, 9.7ms
image 6/304 /content/rock-paper-scissors/test/images/20220216_221819_jpg.rf.295ebb583293f91f74e1700f0ab0639a.jpg: 256x256 1 Rock, 9.7ms
image 7/304 /content/rock-paper-scissors/test/images/20220216_221856_jpg.rf.c551cb3856f480cba36d6aa58e3300cd.jpg: 256x256 1 Paper, 9.7ms
image 8/30

In [56]:
for i, r in enumerate(result):
    boxes = r.boxes  # Boxes object for bbox outputs
    img_id = r.path

    print("boxes = \n", boxes)
    print("img_id = \n", img_id)
    # print("probs = \n", probs)
    if i >= 10:
        break


boxes = 
 ultralytics.engine.results.Boxes object with attributes:

cls: tensor([], device='cuda:0')
conf: tensor([], device='cuda:0')
data: tensor([], device='cuda:0', size=(0, 6))
id: None
is_track: False
orig_shape: (640, 640)
shape: torch.Size([0, 6])
xywh: tensor([], device='cuda:0', size=(0, 4))
xywhn: tensor([], device='cuda:0', size=(0, 4))
xyxy: tensor([], device='cuda:0', size=(0, 4))
xyxyn: tensor([], device='cuda:0', size=(0, 4))
img_id = 
 /content/rock-paper-scissors/test/images/10e0gvm_jpg.rf.3b68a834fab647f30a57fc3ea92d4cd2.jpg
boxes = 
 ultralytics.engine.results.Boxes object with attributes:

cls: tensor([], device='cuda:0')
conf: tensor([], device='cuda:0')
data: tensor([], device='cuda:0', size=(0, 6))
id: None
is_track: False
orig_shape: (640, 640)
shape: torch.Size([0, 6])
xywh: tensor([], device='cuda:0', size=(0, 4))
xywhn: tensor([], device='cuda:0', size=(0, 4))
xyxy: tensor([], device='cuda:0', size=(0, 4))
xyxyn: tensor([], device='cuda:0', size=(0, 4))
img_

## 3. Hymenoptera_data Computer Vision Project: Custom dataset

### hymenoptera_data 자료 다운로드

In [59]:
# !pip install labelImg

url =  "https://download.pytorch.org/tutorial/hymenoptera_data.zip"
filename = "hymenoptera_data.zip"


try:
    response = requests.get(url, stream=True)

    with open(filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=100):
            f.write(chunk)

    print(f"File downloaded successfully to {filename}")

except requests.exceptions.RequestException as e:
    print(f"Error downloading the file: {e}")

## bounding box with roboflow or labelImg

File downloaded successfully to hymenoptera_data.zip


In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="HHKXlDyiIfBO33SpID7d")
project = rf.workspace("firedetection-ysypi").project("ant_bee-mx5dw")
version = project.version(2)
dataset = version.download("yolov11")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ant_bee-2 in yolov11:: 100%|██████████| 88/88 [00:00<00:00, 8483.08it/s]


### Label images dataset
- bounding box와 클래스를 포함한 자료 (예, roboflow, labelImg)
- yaml 파일 만들기

In [5]:
data = {
    'train' : '/content/ant_bee/train/images',
    'val' : '/content/ant_bee/valid/images',
    'test' : '/content/ant_bee/test/images',
    'names' : ["ant", "bee"],
    'nc' : 2
}

with open('/content/ant_bee/data.yaml', 'w') as f:
    yaml.dump(data, f)

In [6]:
model = ultralytics.YOLO('yolo11n.pt')

print(model.names) # pretrained model trained using coco dataset

{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

### Yolo v11 hymenoptera data 객체검출 학습

In [10]:
model.train(data = '/content/ant_bee/data.yaml', epochs = 20, patience = 100, imgsz = 256)

Ultralytics 8.3.172 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/ant_bee/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/runs/detect/train/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, p

train: Scanning /content/ant_bee/train/labels.cache... 33 images, 0 backgrounds, 0 corrupt: 100%|██████████| 33/33 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 422.4±273.0 MB/s, size: 46.4 KB)


val: Scanning /content/ant_bee/valid/labels.cache... 3 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3/3 [00:00<?, ?it/s]


...

                   all          3          3    0.00223       0.75    0.00884    0.00499



20 epochs completed in 0.004 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 5.4MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 5.4MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.172 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 40.99it/s]


                   all          3          3    0.00244       0.75     0.0112    0.00641
                   ant          2          2    0.00242        0.5    0.00705    0.00211
                   bee          1          1    0.00246          1     0.0153     0.0107
Speed: 0.1ms preprocess, 4.2ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a7b15d519d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [12]:
model_path = '/content/runs/detect/train2/weights/best.pt'
model = ultralytics.YOLO(model_path)

In [13]:
result = model(source = '/content/ant_bee/test/images', save = True)


image 1/2 /content/ant_bee/test/images/153320619_2aeb5fa0ee_jpg.rf.aa8b37b12abcbda3ad6a58e356b1c434.jpg: 256x256 (no detections), 13.3ms
image 2/2 /content/ant_bee/test/images/6a00d8341c630a53ef00e553d0beb18834-800wi_jpg.rf.ecc94648169630a9ddf8e7c78b4947b1.jpg: 256x256 (no detections), 11.1ms
Speed: 1.1ms preprocess, 12.2ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 256)
Results saved to runs/detect/predict2
